[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/contents/gray_SQ/gray_SQ_companded.ipynb)

[![Colab](https://badgen.net/badge/Launch/on%20Google%20Colab/blue?icon=notebook)](https://colab.research.google.com/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/contents/gray_SQ/gray_SQ_companded.ipynb)

# Gray-scale Image compression Using a Companded Uniform Midtread Scalar Quantizer

In [ ]:
%%bash
if [ -d "$HOME/repos" ]; then
    echo "\"$HOME/repos\" exists"
else
    mkdir ~/repos
    echo Created $HOME/repos
fi

In [ ]:
%%bash
if [ -d "$HOME/repos/scalar_quantization" ]; then
    cd $HOME/repos/scalar_quantization
    echo "$HOME/repos/scalar_quantization ... "
    git pull 
else
    cd $HOME/repos
    git clone https://github.com/vicente-gonzalez-ruiz/scalar_quantization.git
fi

In [ ]:
%%bash
if [ -d "$HOME/repos/MRVC" ]; then
    cd $HOME/repos/MRVC
    echo "$HOME/repos/MRVC ... "
    git pull 
else
    cd $HOME/repos
    git clone https://github.com/Sistemas-Multimedia/MRVC.git
fi

In [ ]:
%%bash
if [ -d "$HOME/repos/image_IO" ]; then
    cd $HOME/repos/image_IO
    echo "$HOME/repos/image_IO ... "
    git pull 
else
    cd $HOME/repos
    git clone https://github.com/vicente-gonzalez-ruiz/image_IO.git
fi

In [ ]:
%%bash
if [ -d "$HOME/repos/information_theory" ]; then
    cd $HOME/repos/image_IO
    echo "$HOME/repos/information_theory ... "
    git pull 
else
    cd $HOME/repos
    git clone https://github.com/vicente-gonzalez-ruiz/information_theory.git
fi

In [ ]:
#!ln -sf ~/MRVC/src/logging_config.py .
!ln -sf ~/repos/scalar_quantization/quantizer.py .
!ln -sf ~/repos/scalar_quantization/deadzone_quantization.py .
!ln -sf ~/repos/scalar_quantization/companded_quantization.py .
!ln -sf ~/repos/information_theory/distortion.py .
!ln -sf ~/repos/information_theory/information.py .
!ln -sf ~/repos/image_IO/image_1.py .
!ln -sf ~/repos/image_IO/logging_config.py .

In [ ]:
try:
    import matplotlib.pyplot as plt
except:
    !pip install matplotlib
    import matplotlib
    import matplotlib.pyplot as plt
    import matplotlib.axes as ax
    #plt.rcParams['text.usetex'] = True
    #plt.rcParams['text.latex.preamble'] = [r'\usepackage{amsmath}'] #for \text command
%matplotlib inline

try:
    import scipy
except:
    !pip install scipy
    
try:
    import cv2
except:
    !pip install opencv-python
    !pip install opencv-python-headless # Binder compatibility
    import cv2

try:
    import skimage
except:
    !pip install scikit-image
    import skimage
    
try:
    import colored
except:
    !pip install colored
    import colored

import pylab
import math
import numpy as np
from scipy import signal
import cv2
import os
import companded_quantization as quantization
import distortion
#import image_3 as RGB_image
import image_1 as gray_image
import colored
import information

## Configuration

In [ ]:
# Prefix of the gray-scale image to be quantized.

home = os.environ["HOME"]
#fn = home + "/repos/MRVC/images/lena_bw/"
#fn = home + "/repos/MRVC/images/Hommer_bw/"
#fn = home + "/repos/MRVC/images/barb_bw/"
#fn = home + "/repos/MRVC/images/boat_bw/"
#fn = home + "/repos/MRVC/images/crowd_bw/"
#fn = home + "/repos/MRVC/images/fear_bw/"
fn = home + "/repos/MRVC/images/goldhill_bw/"
#fn = home + "/repos/MRVC/images/zelda_bw/"
!ls -l {fn}

quantizer = quantization.Companded_Quantizer

#RGB_image.write = RGB_image.debug_write # faster
#RGB_image.write = RGB_image.write # higher compression

gray_image.write = gray_image.debug_write # faster
#gray_image.write = gray_image.write # higher compression

## Bins and representation levels

In [ ]:
QSS = 1024 # Quantization Step Size
Q = quantizer(Q_step=QSS, min_val=-32768, max_val=32768)
print("decision_levels =", Q.get_decision_levels().astype(np.int32))
print("representation_levels =", Q.get_representation_levels())

## Quantization indexes and their reconstructions

In [ ]:
Q.quantize(Q.get_representation_levels())

In [ ]:
Q.dequantize(Q.quantize(Q.get_representation_levels()))

As it can be seen:
1. We can produce the output 0 (midtread).
2. The representation levels are in the middle point of the bins.
3. $\Delta$ grows exponientially with $|k|$.

## Read the image and show it

In [ ]:
img = gray_image.read(fn, 0)
img.dtype

In [ ]:
print(img.max(), img.min())
gray_image.show(img, fn + "000.png")

## Show the histogram

In [ ]:
histogram, bin_edges = np.histogram(img, bins=256, range=(0, 255))

plt.figure()
plt.title("Histogram")
plt.xlabel("Intensity")
plt.ylabel("Count")
#plt.xlim([0.0, 1.0])  # <- named arguments do not work here
plt.plot(bin_edges[0:-1], histogram)  # <- or here

## Shift the pixel intensities

The intensities are shifted because:
1. The quantizer is symmetic arround of 0.
2. If the histogram shows more counts in the center of the dynamic range, we will generate less quantization error.

In [ ]:
img_128 = img.astype(np.int16) - 128
histogram, bin_edges = np.histogram(img_128, bins=256, range=(-128, 127))

plt.figure()
plt.title("Histogram")
plt.xlabel("Intensity")
plt.ylabel("Count")
#plt.xlim([0.0, 1.0])  # <- named arguments do not work here
plt.plot(bin_edges[0:-1], histogram) 

## Visualize a quantization result

In [ ]:
QSS = 128 # Quantization Step Size
Q = quantizer(Q_step=QSS, min_val=-128, max_val=127)
y, k = Q.quan_dequan(img_128)
print("Used quantization indexes:", np.unique(k))
gray_image.show(k + 128, f"{quantization.name} $\\Delta={QSS}$ (quantization indexes + 128)")
gray_image.show_normalized(k, f"{quantization.name} $\\Delta={QSS}$ (normalized quantization indexes)")
gray_image.show(y + 128, f"{quantization.name} $\\Delta={QSS}$ (dequantized image)")
print("MSE =", distortion.MSE(img, y))
print("SSIM =", distortion.SSIM(img, y))
print("entropy =", information.entropy(k.flatten()))

## RD curve

In [ ]:
def RD_curve(img, Q_steps, quantizer):
    points = []
    for Q_step in Q_steps:
        Q = quantizer(Q_step=Q_step, min_val=0, max_val=255)
        y, k = Q.quan_dequan(img)
        k = k.astype(np.uint8) # Only positive components can be written in an PNG file
        print("Quantization indexes: ", np.unique(k))
        rate = gray_image.write(k, "/tmp/" + str(Q_step) + '_', 0)*8/(k.shape[0]*k.shape[1])
        _distortion = distortion.RMSE(img, y)
        points.append((rate, _distortion))
        print(f"q_step={Q_step:>3}, rate={rate:>7} bits/pixel, distortion={_distortion:>6.1f}")
    return points

Q_steps = [2**i for i in range(15, -1, -1)] # Quantization steps (simulating bit-plane encoding)
print(Q_steps)

RD_points = RD_curve(img_128, Q_steps, quantizer)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RD_points), c='m', marker='x', label=fn, linestyle="dotted")
pylab.title(f"Rate/Distortion Performance ({quantization.name})")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()

In [ ]:
with open("gray_companded_SQ.txt", 'w') as f:
    for item in RD_points:
        f.write(f"{item[0]}\t{item[1]}\n")

## What happens if we increase the granuality?
Let's see the effect of using a finer quantization step (size).

In [ ]:
Q_steps = range(32768, 0, -64)

In [ ]:
def _(a, cmap, vmin, vmax):
    pass
plt.show = print
plt.imshow = _
RD_points_finer = RD_curve(img_128, Q_steps, quantizer)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RD_points), c='m', marker='x', label=f"Using powers of 2", linestyle="dotted")
pylab.scatter(*zip(*RD_points_finer), c='g', marker='o', label=f"Using more quantization steps", s=5)
pylab.title(f"Rate/Distortion Performance ({quantization.name}, {fn})")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
pylab.legend(loc='upper right')
pylab.show()

In [ ]:
import time
while True:
    time.sleep(1)